In [1]:
try:
    spark.stop()
except:
    pass

# Using findspark to find automatically the spark folder
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession, types
from pyspark.sql.functions import explode
from pyspark.sql.functions import split, col
from pyspark.sql.functions import last,max

In [3]:
spark = SparkSession.builder.appName("phase1").getOrCreate()

In [4]:
line = spark.readStream.format("socket").option("host", "localhost").option("port", 9998).load()
phone = spark.readStream.format("socket").option("host","192.168.1.142").option("port",9000).load()

In [5]:
point = line.select(explode(split(line.value," ")).alias("id"))
point = point.withColumn('id', split(point["id"], '\,')[2].cast(types.DoubleType()))
point = point.withColumn('lat', split(point["id"], '\,')[1].cast(types.DoubleType()))
point = point.withColumn('lon', split(point["id"], '\,')[0].cast(types.DoubleType()))

In [6]:
ppoint = phone.select(explode(split(phone.value," ")).alias("id"))
ppoint = ppoint.withColumn('maxlat', split(ppoint["id"], '\,')[5].cast(types.DoubleType()))
ppoint = ppoint.withColumn('minlat', split(ppoint["id"], '\,')[4].cast(types.DoubleType()))
ppoint = ppoint.withColumn('maxlon', split(ppoint["id"], '\,')[3].cast(types.DoubleType()))
ppoint = ppoint.withColumn('minlon', split(ppoint["id"], '\,')[2].cast(types.DoubleType()))
ppoint = ppoint.withColumn('timestamp', split(ppoint["id"], '\,')[1].cast(types.LongType()))
ppoint = ppoint.withColumn('id', split(ppoint["id"], '\,')[0])
ppoint = ppoint.groupby('id').agg(max('timestamp'), last('minlat'), last('maxlat'), last('minlon'), last('maxlon')).toDF('id','time','minlat','maxlat','minlon','maxlon').drop('time')
t1 = ppoint.writeStream.queryName("t1").outputMode("complete").format("memory").start()

In [7]:
t2 = point.join(spark.sql("select * from t1"),'id').groupby('id').count()

In [8]:
queryexe = t2.writeStream.outputMode('complete').format('console').start()
queryexe.awaitTermination()
#query2 = latest.writeStream.outputMode('complete').format("console").start()
#spark.streams.awaitAnyTermination()

AnalysisException: "'writeStream' can be called only on streaming Dataset/DataFrame;"